In [1]:
import re
import os

def normalize_msigdb_name(name):
    
    # Remove the source prefix (everything before and including '::')
    if "::" in name:
        name = name.split("::", 1)[1]
    
    # Remove specific collection prefixes to find the core name.
    prefixes_to_strip = ["GOBP_", "REACTOME_", "KEGG_", "BIOCARTA_", "PID_"]
    for prefix in prefixes_to_strip:
        if name.startswith(prefix):
            name = name[len(prefix):]
            break
            
    return name.upper()

def normalize_lncrna_name(name):

    # Remove prefix (e.g., LNCRNA_GO::)
    if "::" in name:
        name = name.split("::", 1)[1]
    
    # Remove GO IDs or Reactome IDs at the end (e.g., (GO:0051084), R-HSA-72306)
    name = re.sub(r'\s*\(GO:\d+\)', '', name)
    name = re.sub(r'\s*R-HSA-[\d\w-]+', '', name)
    
    # Replace non-alphanumeric characters (like spaces, quotes, hyphens) with underscores
    name = re.sub(r'[^a-zA-Z0-9]+', '_', name)
    
    # Remove leading/trailing underscores and convert to uppercase
    return name.strip('_').upper()

def process_files(msigdb_file, lncrna_file, output_file):
    print(f"Reading {msigdb_file}...")
    msigdb_sets = {}
    
    try:
        with open(msigdb_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 3:
                    original_name = parts[0]
                    # Gene list is the 3rd column (index 2)
                    genes = set(parts[2].split(','))
                    
                    # Store by normalized name for easy lookup
                    norm_name = normalize_msigdb_name(original_name)
                    msigdb_sets[norm_name] = genes
    except FileNotFoundError:
        print(f"Error: Could not find {msigdb_file}")
        return

    print(f"Processing {lncrna_file} and enriching...")
    enriched_count = 0
    
    try:
        with open(lncrna_file, 'r', encoding='utf-8') as f_in, \
             open(output_file, 'w', encoding='utf-8') as f_out:
            
            for line in f_in:
                parts = line.strip().split('\t')
                if len(parts) >= 3:
                    lnc_name_full = parts[0]
                    lnc_source = parts[1]
                    lnc_genes = set(parts[2].split(','))
                    
                    # Normalize the lncRNA set name to find its match
                    norm_name = normalize_lncrna_name(lnc_name_full)
                    
                    # Look for the matching gene set in MSigDB
                    msig_genes = msigdb_sets.get(norm_name)
                    
                    if msig_genes:
                        # Union the genes (LncRNA targets + MSigDB genes)
                        enriched_genes = lnc_genes.union(msig_genes)
                        enriched_gene_str = ','.join(sorted(list(enriched_genes)))
                        f_out.write(f"{lnc_name_full}\t{lnc_source}\t{enriched_gene_str}\n")
                        enriched_count += 1
                    else:
                        # If no match found, write the original line
                        f_out.write(line)
                else:
                    f_out.write(line)
                    
        print(f"Success! Enriched {enriched_count} sets.")
        print(f"Output saved to: {output_file}")

    except FileNotFoundError:
        print(f"Error: Could not find {lncrna_file}")

# Run the processing
if __name__ == "__main__":
    process_files('msigdb_sets.txt', 'lncrna_sets.txt', 'enriched_lncrna_sets.txt')

Reading msigdb_sets.txt...
Processing lncrna_sets.txt and enriching...
Success! Enriched 5958 sets.
Output saved to: enriched_lncrna_sets.txt


In [2]:
import re

def load_gene_set(filepath, is_msigdb=False):
    """Reads a file and returns a dictionary {Normalized_Name: Set_of_Genes}"""
    gene_data = {}
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 3:
                    name = parts[0]
                    genes = set(parts[2].split(','))
                    
                    # Normalize name to match the logic used in enrichment
                    if "::" in name:
                        name = name.split("::", 1)[1]
                    name = re.sub(r'\s*\(GO:\d+\)', '', name)
                    name = re.sub(r'\s*R-HSA-[\d\w-]+', '', name)
                    name = re.sub(r'[^a-zA-Z0-9]+', '_', name)
                    name = name.strip('_').upper()
                    
                    if is_msigdb:
                        # MSigDB specific prefix stripping
                        for prefix in ["GOBP_", "REACTOME_", "KEGG_", "BIOCARTA_", "PID_"]:
                            if name.startswith(prefix):
                                name = name[len(prefix):]
                                break
                    
                    gene_data[name] = genes
        return gene_data
    except FileNotFoundError:
        print(f"Error: File not found - {filepath}")
        return {}

def verify_results():
    print("Loading files for verification...")
    # Load original data
    msigdb_data = load_gene_set('msigdb_sets.txt', is_msigdb=True)
    lncrna_data = load_gene_set('lncrna_sets.txt', is_msigdb=False)
    
    # Load the output data we want to verify
    # Note: We treat the output like LncRNA data since it keeps the LncRNA names
    output_data = load_gene_set('enriched_lncrna_sets.txt', is_msigdb=False)

    success_count = 0
    failure_count = 0
    no_match_count = 0

    print(f"\nVerifying {len(output_data)} sets...")

    for norm_name, output_genes in output_data.items():
        # Get the original genes from the LncRNA file
        original_lnc_genes = lncrna_data.get(norm_name, set())
        
        # Get the genes involved in the enrichment (from MSigDB)
        msigdb_genes = msigdb_data.get(norm_name)

        if msigdb_genes:
            # Calculate what the result SHOULD be
            expected_genes = original_lnc_genes.union(msigdb_genes)
            
            # Compare actual output vs expected
            if output_genes == expected_genes:
                success_count += 1
            else:
                print(f"MISMATCH for: {norm_name}")
                print(f"  Expected count: {len(expected_genes)}")
                print(f"  Found count:    {len(output_genes)}")
                failure_count += 1
        else:
            # If no MSigDB match existed, the genes should remain unchanged
            if output_genes == original_lnc_genes:
                no_match_count += 1
            else:
                print(f"ERROR: Unmatched set changed unexpectedly: {norm_name}")
                failure_count += 1

    print("\n" + "="*30)
    print("VERIFICATION RESULTS")
    print("="*30)
    print(f"Successfully Enriched:  {success_count}")
    print(f"No Match (Unchanged):   {no_match_count}")
    print(f"Failed/Mismatched:      {failure_count}")
    
    if failure_count == 0:
        print("\n✅ SUCCESS: The enrichment process was verified explicitly.")
    else:
        print("\n❌ WARNING: Some sets did not match expectations.")

if __name__ == "__main__":
    verify_results()

Loading files for verification...

Verifying 7685 sets...

VERIFICATION RESULTS
Successfully Enriched:  5901
No Match (Unchanged):   1784
Failed/Mismatched:      0

✅ SUCCESS: The enrichment process was verified explicitly.


In [3]:
def extract_all_genes(input_file, output_file):
    print(f"Reading genes from {input_file}...")
    unique_genes = set()
    
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                # Ensure the line has at least 3 columns (Name, Source, Genes)
                if len(parts) >= 3:
                    # The gene list is the 3rd column (index 2)
                    gene_list = parts[2].split(',')
                    
                    # Add genes to the set (automatically handles duplicates)
                    # We strip whitespace just in case
                    unique_genes.update(g.strip() for g in gene_list if g.strip())
        
        # Sort the genes alphabetically
        sorted_genes = sorted(list(unique_genes))
        
        # Write to output file
        with open(output_file, 'w', encoding='utf-8') as f_out:
            for gene in sorted_genes:
                f_out.write(gene + '\n')
                
        print(f"Success! Found {len(sorted_genes)} unique genes.")
        print(f"Saved list to: {output_file}")

    except FileNotFoundError:
        print(f"Error: Could not find file '{input_file}'")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Input filename (the result from the previous step)
    input_filename = 'enriched_lncrna_sets.txt'
    
    # Output filename
    output_filename = 'all_unique_genes.txt'
    
    extract_all_genes(input_filename, output_filename)

Reading genes from enriched_lncrna_sets.txt...
Success! Found 21115 unique genes.
Saved list to: all_unique_genes.txt


In [5]:
def filter_genesets(input_file, output_file, min_size=10, max_size=500):
    print(f"Filtering {input_file}...")
    kept_count = 0
    removed_count = 0
    
    try:
        with open(input_file, 'r', encoding='utf-8') as f_in, \
             open(output_file, 'w', encoding='utf-8') as f_out:
            
            for line in f_in:
                parts = line.strip().split('\t')
                
                # Ensure the line has the expected format (Name, Source, Genes)
                if len(parts) >= 3:
                    # Gene list is in the 3rd column (index 2)
                    genes = parts[2].split(',')
                    # Filter out empty strings just in case
                    genes = [g for g in genes if g.strip()]
                    
                    count = len(genes)
                    
                    # Logic: Keep if size is >= 10 AND <= 500
                    if min_size <= count <= max_size:
                        f_out.write(line)
                        kept_count += 1
                    else:
                        removed_count += 1
                else:
                    # Handle malformed lines (optional: skip or write as is)
                    # Usually better to skip if strict filtering is needed
                    removed_count += 1

        print(f"Filtering complete.")
        print(f" - Sets kept (10-500 genes): {kept_count}")
        print(f" - Sets removed: {removed_count}")
        print(f"Output saved to: {output_file}")

    except FileNotFoundError:
        print(f"Error: Could not find file '{input_file}'")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Define file names
    input_filename = 'enriched_lncrna_sets.txt'
    output_filename = 'filtered_enriched_sets.txt'
    
    # Run the filter
    filter_genesets(input_filename, output_filename, min_size=10, max_size=500)

Filtering enriched_lncrna_sets.txt...
Filtering complete.
 - Sets kept (10-500 genes): 6182
 - Sets removed: 1561
Output saved to: filtered_enriched_sets.txt
